# Timeseries download - work in progress

So far, this script only downloads solar and wind timeserie from several european TSOs

#####Loading some python libraries and defining downloads path inside the notebook's directory on the users computer

In [1]:
import urllib
import os
from dateutil import rrule
from datetime import datetime

path = 'downloads/'
if not os.path.exists(path): os.makedirs(path)

#####This is a python dictionary containing input parameters needed to generate the URLs belonging to the data sources

In [2]:
conf = {
    'Amprion': {
        'wind': {
            'url_template': 'http://preview.amprion.de/applications/applicationfiles/winddaten.php?mode=download&format=csv&start={u_start:%Y.%m.%d}&end={u_end:%Y.%m.%d}',
            'bundle': 'complete',
            'start': '2006-01-03',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://amprion.de/applications/applicationfiles/PV_einspeisung.php?mode=download&format=csv&start={u_start:%Y.%m.%d}&end={u_end:%Y.%m.%d}',
            'bundle': 'complete',
            'start': '2010-01-07',
            'end': '2015-01-01',
        }
    }    
    ,'CEPS': {
        'wind_pv': {
            'url_template': 'http://www.ceps.cz/_layouts/15/Ceps/_Pages/GraphData.aspx?mode=xlsx&from={u_start:%m.%d.%Y}%2012:00:00%20AM&to={u_end:%m/%d/%Y}%2011:59:59%20PM&hasinterval=False&sol=26&lang=ENG&agr=QH&fnc=SUM&ver=RT&para1=all&',
            'bundle': 'complete',
            'start': '2012-01-01',
            'end': '2015-01-01',
        }
    }    
    ,'TransnetBW': {
        'wind': {
            'url_template': 'https://www.transnetbw.de/de/kennzahlen/erneuerbare-energien/windenergie?app=wind&activeTab=csv&selectMonatDownload={month}&view=1&download=true',
            'bundle': 'special',
            'start': '2010-01-01',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'https://www.transnetbw.de/de/kennzahlen/erneuerbare-energien/fotovoltaik?app=solar&activeTab=csv&selectMonatDownload={month}&view=1&download=true',
            'bundle': 'special',
            'start': '2011-01-01',
            'end': '2015-01-01',
        }
    }
    ,'TenneT': {
        'wind': {
            'url_template': 'http://www.tennettso.de/site/de/phpbridge?commandpath=Tatsaechliche_und_prognostizierte_Windenergieeinspeisung%2FmonthDataSheetCsv.php&querystring=monat%3D{year}-{month:02d}&contenttype=text%2Fx-csv',
            'bundle': 'MONTHLY',
            'start': '2005-07-13',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://www.tennettso.de/site/de/phpbridge?commandpath=Tatsaechliche_und_prognostizierte_Solarenergieeinspeisung%2FmonthDataSheetCsv.php&sub=total&querystring=monat%3D{year}-{month:02d}&contenttype=text%2Fx-csv',
            'bundle': 'MONTHLY',
            'start': '2010-01-01',
            'end': '2015-01-01',
        }
    }
    ,'50hertz': {
        'wind': {
            'url_template': 'http://ws.50hertz.com/web01/api/WindPowerForecast/DownloadFile?fileName={year}.csv&callback=?',
            'bundle': 'YEARLY',
            'start': '2005-01-01',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://ws.50hertz.com/web01/api/PhotovoltaicForecast/DownloadFile?fileName={year}.csv&callback=?',
            'bundle': 'YEARLY',
            'start': '2012-01-01',
            'end': '2015-01-01',
        }
    }
    ,'RTE': {
        'wind_pv': {
            'url_template': 'http://clients.rte-france.com/servlets/RealProdServlet?annee={year}',
            'bundle': 'YEARLY',
            'start': '2014-01-01',
            'end': '2015-01-01', 
        }
    }
    ,'Svenska_Kraftnaet': {
        'wind_pv_1': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot{year}.xls',
            'bundle': 'YEARLY',
            'start': '2002-01-01',
            'end': '2009-01-01',
        }
        ,'wind_pv_2': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot201001-06.xls',
            'bundle': 'YEARLY',
            'start': '2010-01-01',
            'end': '2010-01-01',
        }
        ,'wind_pv_3': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot2010-07-12.xls',
            'bundle': 'YEARLY',
            'start': '2010-01-01',
            'end': '2010-01-01',
        }
        ,'wind_pv_4': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n-fot2011-01-12.xls',
            'bundle': 'YEARLY',
            'start': '2011-01-01',
            'end': '2011-01-01',
        }
        ,'wind_pv_5': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot{year}-01-12.xls',
            'bundle': 'YEARLY',
            'start': '2012-01-01',
            'end': '2014-01-01',
        }
    }
    ,'OeMag': {
        'wind_1': {
            'url_template': 'http://www.oem-ag.at/fileadmin/user_upload/Dokumente/statistik/winderzeugung/winderzeugung_{year}.xls',
            'bundle': 'YEARLY',
            'start': '2003-01-01',
            'end': '2011-01-01',
        }
        ,'wind_2': {
            'url_template': 'http://www.oem-ag.at/fileadmin/user_upload/Dokumente/statistik/winderzeugung/winderzeugung_{year}.xlsx',
            'bundle': 'YEARLY',
            'start': '2012-01-01',
            'end': '2012-01-01',
        }
    }            
}

#####Here we iterate over the sources and technology (wind/solar) entries specified above and download the data for all periods (depending on whats specified under 'bundle' either years, days or months) 

In [6]:
def download1(url_template, period, source, tech):
    return
    name = source+'_'+tech+'_'+period.strftime('%Y-%m-%d')
    print('attempting download of ', name)
    full_url = url_template.format(year = period.year, month = period.month, day = period.day)
    print(full_url)
    file = urllib.request.urlopen(full_url)
    data = file.read()
    with open(path+name+'.csv', 'wb') as output_file:
        output_file.write(data)
    print('done')
    
def download2(url_template, count, source, tech):
    return
    name = source+'_'+tech+'_'+str(count)
    print('attempting download of ', name)
    full_url = url_template.format(month = count)
    print(full_url)
    file = urllib.request.urlopen(full_url)
    data = file.read()
    with open(path+name+'.csv', 'wb') as output_file:
        output_file.write(data)
    print('done')

def download3(url_template, start, end, source, tech):
    name = source+'_'+tech+'_'+start.strftime('%Y-%m-%d')+'_'+end.strftime('%Y-%m-%d')
    print('attempting download of ', name)
    full_url = url_template.format(u_start = start, u_end = end)
    print(full_url)
    file = urllib.request.urlopen(full_url)
    data = file.read()
    with open(path+name+'.csv', 'wb') as output_file:
        output_file.write(data)
    print('done')
    
for source, tech in conf.items():
    for tech, param in tech.items():
        if source == 'TransnetBW': #TransnetBW uses a different format for the months, this requires a different approach
            for count in range(0,70): #iterates over range from 0 to 69
                download2(param['url_template'], count, source, tech)
            continue #jump to the next tech  
        start = datetime.strptime(param['start'], '%Y-%m-%d').date()
        end = datetime.strptime(param['end'], '%Y-%m-%d').date()
        if param['bundle'] == 'complete':
            download3(param['url_template'], start, end, source, tech)
        if param['bundle'] == 'YEARLY':
            for period in rrule.rrule(rrule.YEARLY, dtstart=start, until=end):
                download1(param['url_template'], period, source, tech)
        if param['bundle'] == 'MONTHLY':
            for period in rrule.rrule(rrule.MONTHLY, dtstart=start, until=end):
                download1(param['url_template'], period, source, tech)               
        if param['bundle'] == 'DAILY':
            for period in rrule.rrule(rrule.DAILY, dtstart=start, until=end):
                download1(param['url_template'], period, source, tech)

attempting download of  CEPS_wind_pv_2012-01-01_2015-01-01
http://www.ceps.cz/_layouts/15/Ceps/_Pages/GraphData.aspx?mode=xlsx&from=01.01.2012%2012:00:00%20AM&to=01/01/2015%2011:59:59%20PM&hasinterval=False&sol=26&lang=ENG&agr=QH&fnc=SUM&ver=RT&para1=all&
success
attempting download of  Amprion_pv_2010-01-07_2015-01-01
http://amprion.de/applications/applicationfiles/PV_einspeisung.php?mode=download&format=csv&start=2010.01.07&end=2015.01.01
success
attempting download of  Amprion_wind_2006-01-03_2015-01-01
http://preview.amprion.de/applications/applicationfiles/winddaten.php?mode=download&format=csv&start=2006.01.03&end=2015.01.01
success
